In [27]:
!pip install --no-cache-dir transformers sentencepiece
!pip install pip install pytelegrambotapi
!pip install datasets
!pip install evaluate
!mkdir img
#!unzip archivet.zip -d dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘img’: File exists


In [28]:
import telebot
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import numpy as np
import cv2
import csv
import evaluate
import time

In [29]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten', use_fast=False)
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
processor1 = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten', use_fast=False)
model1 = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
processor2 = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten', use_fast=False)
model2 = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-handwritten')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
def model_answer(path):
    mes = ""
    try:
      image = cv2.imread(path)
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      blur = cv2.medianBlur(gray, 5)
      thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,8)
      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
      dilate = cv2.dilate(thresh, kernel, iterations=6)
      cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cnts = cnts[0] if len(cnts) == 2 else cnts[1]
      #cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
      cnts = cnts[-1:0:-1]
      for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        im_pil = Image.fromarray(ROI)
        pixel_values = processor(images=im_pil, return_tensors="pt").pixel_values

        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        generated_text = generated_text.replace('.', '')
        generated_text = generated_text.replace('#', '')
        generated_text = generated_text.replace('_', '')
        generated_text = generated_text.replace('0', '')
        generated_text = generated_text.replace('"', '')
        if len(generated_text) < 2 and generated_text != 'a':
          generated_text = ""
        if generated_text != "":
          mes += generated_text + "\n"
      if mes == "":
        mes = "Не нашел текста :("
    except Exception as e:
      print(e)
      mes = "Что-то пошло не так :("
    return mes


bot = telebot.TeleBot("")

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, "Отправьте картинку")

@bot.message_handler(content_types=['text'])
def other_text(message):
    bot.send_message(message.chat.id, "Отправьте картинку")

@bot.message_handler(content_types=['photo'])
def get_img(message):
    msg = bot.send_message(message.chat.id, "Идет обработка...")
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    download_file = bot.download_file(file_info.file_path)
    path = f'./img/img{message.chat.id}.{file_info.file_path.split(".")[-1]}'
    with open(path, 'wb') as file:
        file.write(download_file)
    bot.edit_message_text(chat_id=message.chat.id, message_id=msg.message_id, text=model_answer(path))
    bot.send_message(message.chat.id, "Отправьте картинку")

bot.polling(non_stop=True, interval=0)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
with open("written_name_test_v2.csv") as f:
    reader = csv.reader(f)
    check_dict = dict((rows[0],rows[1]) for rows in reader)
predictions = []
references = []
for file in check_dict.keys():
  image = Image.open(f"./dataset/test/{file}").convert("RGB")
  pixel_values = processor(images=image, return_tensors="pt").pixel_values

  generated_ids = model.generate(pixel_values)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  predictions.append(generated_text)
  references.append([check_dict[file]])

In [ ]:
with open("written_name_test_v2.csv") as f:
    reader = csv.reader(f)
    check_dict = dict((rows[0],rows[1]) for rows in reader)
predictions1 = []
references1 = []
for file in check_dict.keys():
  image = Image.open(f"./dataset/test/{file}").convert("RGB")
  pixel_values = processor1(images=image, return_tensors="pt").pixel_values

  generated_ids = model1.generate(pixel_values)
  generated_text = processor1.batch_decode(generated_ids, skip_special_tokens=True)[0]
  predictions1.append(generated_text)
  references1.append([check_dict[file]])

In [ ]:
with open("written_name_test_v2.csv") as f:
    reader = csv.reader(f)
    check_dict = dict((rows[0],rows[1]) for rows in reader)
predictions2 = []
references2 = []
for file in check_dict.keys():
  image = Image.open(f"./dataset/test/{file}").convert("RGB")
  pixel_values = processor2(images=image, return_tensors="pt").pixel_values

  generated_ids = model2.generate(pixel_values)
  generated_text = processor2.batch_decode(generated_ids, skip_special_tokens=True)[0]
  predictions2.append(generated_text)
  references2.append([check_dict[file]])

In [36]:
for i in range(len(predictions)):
  predictions[i] = predictions[i].replace('.', '')
  predictions[i] = predictions[i].replace(' ', '')
  predictions[i] = predictions[i].replace('#', '')
  predictions[i] = predictions[i].replace('_', '')
  predictions[i] = predictions[i].upper()

In [37]:
for i in range(len(predictions1)):
  predictions1[i] = predictions1[i].replace('.', '')
  predictions1[i] = predictions1[i].replace(' ', '')
  predictions1[i] = predictions1[i].replace('#', '')
  predictions1[i] = predictions1[i].replace('_', '')
  predictions1[i] = predictions1[i].upper()

In [38]:
for i in range(len(predictions2)):
  predictions2[i] = predictions2[i].replace('.', '')
  predictions2[i] = predictions2[i].replace(' ', '')
  predictions2[i] = predictions2[i].replace('#', '')
  predictions2[i] = predictions2[i].replace('_', '')
  predictions2[i] = predictions2[i].upper()

In [39]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print("large", results['precisions'][0]*100)

large 10.606060606060606


In [40]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions1, references=references1)
print("base", results['precisions'][0]*100)

base 3.9215686274509802


In [41]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions2, references=references2)
print("small", results['precisions'][0]*100)

small 11.11111111111111


In [44]:
with open("written_name_test_v2.csv") as f:
    reader = csv.reader(f)
    check_dict = dict((rows[0],rows[1]) for rows in reader)
times = []
times1 = []
times2 = []
for file in check_dict.keys():
  image = Image.open(f"./dataset/test/{file}").convert("RGB")
  time1 = time.time()
  pixel_values = processor(images=image, return_tensors="pt").pixel_values

  generated_ids = model.generate(pixel_values)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  time2 = time.time()
  times.append(time2 - time1)
  time1 = time.time()
  pixel_values = processor1(images=image, return_tensors="pt").pixel_values

  generated_ids = model1.generate(pixel_values)
  generated_text = processor1.batch_decode(generated_ids, skip_special_tokens=True)[0]
  time2 = time.time()
  times1.append(time2 - time1)
  time1 = time.time()
  pixel_values = processor2(images=image, return_tensors="pt").pixel_values

  generated_ids = model2.generate(pixel_values)
  generated_text = processor2.batch_decode(generated_ids, skip_special_tokens=True)[0]
  time2 = time.time()
  times2.append(time2 - time1)
print("large", sum(times)/float(len(times)))
print("base", sum(times1)/float(len(times1)))
print("small", sum(times2)/float(len(times2)))

large 14.50441312789917
base 10.60425066947937
small 0.8907990455627441
